# Random forest classifier

In [9]:
from src.data.data_fetcher import get_raw_data
from src.features.feature_engineering import prepare_data
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [10]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

In [11]:
targets = train_a
features_observed = X_train_observed_a
features_estimated = X_train_estimated_a
features_test = X_test_estimated_a

# Call the function with the file paths
X_train, y_train, X_val, y_val, X_test, train_data, val_data = prepare_data(targets, 
                                                                            features_observed, 
                                                                            features_estimated, 
                                                                            features_test)

# Check the prepared DataFrames
X_train.head(1), y_train.head(1), X_val.head(1), y_val.head(1), X_test.head(1), train_data.head(1), val_data.head(1)


(                     absolute_humidity_2m:gm3  air_density_2m:kgm3  \
 2019-12-01 09:00:00                       4.8                1.272   
 
                      ceiling_height_agl:m  clear_sky_energy_1h:J  \
 2019-12-01 09:00:00                456.25                27331.5   
 
                      clear_sky_rad:W  cloud_base_agl:m  dew_or_rime:idx  \
 2019-12-01 09:00:00        17.549999            456.25              0.0   
 
                      dew_point_2m:K  diffuse_rad:W  diffuse_rad_1h:J  ...  \
 2019-12-01 09:00:00      273.150024          5.175       9751.150391  ...   
 
                      sun_azimuth:d  sun_elevation:d  \
 2019-12-01 09:00:00     155.780746          2.32725   
 
                      super_cooled_liquid_water:kgm2  t_1000hPa:K  \
 2019-12-01 09:00:00                            0.25   274.350006   
 
                      total_cloud_cover:p  visibility:m  wind_speed_10m:ms  \
 2019-12-01 09:00:00                100.0   7155.975098               4.

In [12]:
# X_val contains features not present in X_train
# Drop those features from X_val
X_val = X_val.drop(['date_calc'], axis=1)

# Initialize a Random Forest regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the Random Forest model on the cleaned training data
rf_model.fit(X_train, y_train)

# Predict on the cleaned validation set
y_val_pred_rf = rf_model.predict(X_val)

# Evaluate the Random Forest model using MAE on the cleaned validation set
mae_rf = mean_absolute_error(y_val, y_val_pred_rf)
mae_rf


56.88844569892473